## Search for Landsat-8 products using the "eop:track" as a queryable

In [23]:
import lxml.etree as etree
import requests
import cioppy
ciop = cioppy.Cioppy()

In [34]:
def get_params(osd):

    oss_ns = {'a':'http://www.w3.org/2001/XMLSchema', 
          'b':'http://www.w3.org/2001/XMLSchema-instance',
          'c':'http://a9.com/-/opensearch/extensions/time/1.0/',
          'd':'http://www.opengis.net/eop/2.0',
          'e':'http://purl.org/dc/terms/',
          'f':'http://a9.com/-/spec/opensearch/extensions/parameters/1.0/',
          'g':'http://purl.org/dc/elements/1.1/',
          'h':'http://www.terradue.com/opensearch',
          'i':'http://a9.com/-/opensearch/extensions/geo/1.0/',
          'j':'http://a9.com/-/spec/opensearch/1.1/'}
    
    oss_content = etree.fromstring(requests.get(osd).content)
    
    url_template_element = oss_content.xpath('/j:OpenSearchDescription/j:Url[@type="application/atom+xml"]',
                                                 namespaces=oss_ns)[0]
    
    parameters = dict()
    
    for index, parameter in enumerate(url_template_element.xpath('.//f:Parameter', namespaces=oss_ns)):
        
        parameters[parameter.attrib['name']] = {'title' : parameter.attrib['title'], 
                                                'value' : parameter.attrib['value']}
        
        options = []
        for option in parameter.xpath('.//f:Option', namespaces=oss_ns):
            
            options.append(option.attrib['value'])
    
        parameters[parameter.attrib['name']] = {'title' : parameter.attrib['title'], 
                                                'value' : parameter.attrib['value'],
                                                'options' : options}
    return parameters

In [35]:
def get_param_value(osd, os_parameter):

    params = get_params(osd)

    res = None
    
    for index, param in enumerate(params):
        
        if params[param]['value'] == os_parameter:
            
            res = params[param]
            res['name'] = param
     
    return res

Define the Landsat-8 endpoint

In [36]:
osd_url = 'https://catalog.terradue.com/landsat8/description'

Get the Landsat-8 search parameters

In [37]:
ls8_parameters = get_params(osd_url)

Print all the OpenSearch parameters

In [38]:
for key, value in ls8_parameters.iteritems():

    print '%24s' % ls8_parameters[key]['value'],  ls8_parameters[key]['title'] 

              {geo:uid?} The identifier of the resource within the search engine context (local reference)
    {t2:downloadOrigin?} a string that identifies the download origin (keyword, hostname...) to adapt the enclosure. If the parameter is enclosed between [] (e.g. [terradue]), enclosure will be returned only if there is a enclosure found for this source.
       {eop:cloudCover?} A number, set or interval requesting the cloud coverage
            {eop:frame?} A number, set or interval requesting the range of orbit frames
           {startIndex?} index of the first search result desired
            {startPage?} page number of the set of search results desired
       {eop:sensorType?} A string identifying the sensor type
     {eop:accessedFrom?} A string identifying the location from which the resource will be accessed. The catalogue shall return the download location in the enclosure atom link according to the parameter value.
         {t2:landCover?} A number, set or interval reque

Find the parameter associated to **'{eop:wrsLongitudeGrid?}'** and its options

In [41]:
product_type_parameter = get_param_value(osd_url, '{eop:track?}')

In [42]:
product_type_parameter['name']

'track'

Now search for Landsat-8 products creating a dictionary with the search parameters

In [43]:
search_params = dict([(product_type_parameter['name'], 
                       '160')])

In [44]:
search_params

{'track': '160'}

In [47]:
search = ciop.search(end_point=osd_url,
                     params=search_params,
                     output_fields='self,productType,track,enclosure,identifier,wkt,startdate', 
                     model='EOP')

Show the first result returned

In [46]:
search[0]

{'enclosure': 'https://store.terradue.com/download/landsat8/files/v1/LC08_L1GT_160131_20171220_20171224_01_T2',
 'identifier': 'LC08_L1GT_160131_20171220_20171224_01_T2',
 'productType': 'L1GT',
 'self': 'https://catalog.terradue.com/landsat8/search?format=atom&uid=LC08_L1GT_160131_20171220_20171224_01_T2',
 'startdate': '2017-12-20T07:21:38.1610000Z',
 'track': '160',
 'wkt': 'POLYGON((-104.89087 -73.49071,-110.32266 -74.41859,-106.86865 -75.91225,-101.12939 -74.89784,-104.89087 -73.49071))'}

You can of course write the search parameters directly with:


In [48]:
search = ciop.search(end_point=osd_url,
                     params={'track': '160'},
                     output_fields='self,productType,track,enclosure,identifier,wkt,startdate', 
                     model='EOP')

In [49]:
search[0]

{'enclosure': 'https://store.terradue.com/download/landsat8/files/v1/LC08_L1GT_160248_20180801_20180814_01_T2',
 'identifier': 'LC08_L1GT_160248_20180801_20180814_01_T2',
 'productType': 'L1GT',
 'self': 'https://catalog.terradue.com/landsat8/search?format=atom&uid=LC08_L1GT_160248_20180801_20180814_01_T2',
 'startdate': '2018-08-01T08:07:22.5870000Z',
 'track': '160',
 'wkt': 'POLYGON((93.72668 81.78101,98.2809 80.20382,108.40676 80.72759,105.58563 82.40689,93.72668 81.78101))'}